# SIP Party to `sipp` #

This demonstrates how to make calls to and from a sipp script without media using SIP Party.

## A: Client SIP Party, server `sipp`.

### Prerequisite steps

You need a working copy of `sipp`. 

This example works with the basic `sipp` server scenario:

    $ sipp -sn uas
    
If you want to see the message flows, switch the level of the `'sipparty.transport.messages'` logger to `logging.INFO`.

In [ ]:
import logging

transport_messages_logger = logging.getLogger('sipparty.transport.messages')
transport_messages_logger.setLevel(logging.WARNING)

### A1: Set up the the URIs

Since it's a `sipp` instance we're talking to it doesn't have a real AOR, so we'll just use it's contact IP address as the host, which saves passing that in as a separate step.

In [ ]:
remote_uri = 'sip:sipp_client@<your sipp host\'s IP here>.'
remote_uri = 'sip:sipp_client@192.168.0.106'
my_uri = 'sip:me@my-domain.com'

### A2: Create the local party

Notice that we don't need to specify local address parameters. `sipparty` will use sensible defaults.

In [ ]:
from sipparty.parties import NoMediaSimpleCallsParty
p1 = NoMediaSimpleCallsParty(uri=my_uri)

### A3: Send the `INVITE` to `sipp`

SIP Party does most of its work asynchronously, but via the `sipparty.fsm.FSM` class provides a synchronization mechanism in the form of the `waitForStateCondition()` method. We can use that to wait for the dialog to be set up.

In [ ]:
dlg = p1.invite(remote_uri)
dlg.waitForStateCondition(
    lambda st: st not in (dlg.States.Initial, dlg.States.InviteSent))

### A4: Terminate

And then simply terminate the dialog when you are ready.

In [ ]:
dlg.terminate()
dlg.waitForStateCondition(lambda st: st == dlg.States.Terminated)